In [1]:
from braket.devices import LocalSimulator

device = LocalSimulator("default")

In [97]:
from braket.circuits import Circuit

# 0: R
# 1: S
# 2: Q

# H(0)
alice = Circuit().h(0).cnot(0, 1).cnot(2, 0).h(2)

print(alice)

T  : |0|1|2|3|
              
q0 : -H-C-X---
        | |   
q1 : ---X-|---
          |   
q2 : -----C-H-

T  : |0|1|2|3|


In [96]:
device.run(alice, shots=1).result().measurement_counts

Counter({'000': 1})